In [3]:
from pathlib import Path
import pandas as pd

csv_files = list(Path("data").rglob("*.csv"))
print("CSV files found:", [str(p) for p in csv_files][:10])

# Если файлов несколько — выбери нужный (поставь правильный индекс)
csv_path = csv_files[0]
print("Using:", csv_path)

df = pd.read_csv(csv_path)
print("Shape:", df.shape)
print("Columns:", list(df.columns))
df.head()


CSV files found: ['data\\Anime_reviews_RU.csv']
Using: data\Anime_reviews_RU.csv
Shape: (73216, 4)
Columns: ['Unnamed: 0', 'Anime', 'Rate', 'Text']


,Unnamed: 0,Anime,Rate,Text
0,0,Нежный укус / Amagami SS,Нейтральный,"после первой арки несколько разочаровался, ожи..."
1,1,Легенда о легендарных героях / Densetsu no Yuu...,Положительный,"Концовка какая-то мутная, ничего не понял. Над..."
2,2,Мобильный воин Гандам: Единорог / Kidou Senshi...,Положительный,"Что тут сказать? Никакого сравнени я с Seed, х..."
3,3,Невиданный цветок / Ano Hi Mita Hana no Namae ...,Положительный,Только что досмотрел и меня переполняют эмоции...
4,4,Принцесса-медуза / Kuragehime,Положительный,"И правда не оторваться. Но, по-моему, то, что ..."


In [4]:
import pandas as pd

# Убираем мусорный столбец
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# Переименуем для удобства
df = df.rename(columns={"Text": "text", "Rate": "label", "Anime": "anime"})

# Удалим пустые
df["text"] = df["text"].astype(str)
df["label"] = df["label"].astype(str)
df = df.dropna(subset=["text", "label"])
df = df[df["text"].str.strip().ne("")]

print("Shape after clean:", df.shape)
print("Labels:", df["label"].value_counts())
df.head()


Shape after clean: (73216, 3)
Labels: label
Положительный    30073
Нейтральный      25263
Отрицательный    17880
Name: count, dtype: int64


,anime,label,text
0,Нежный укус / Amagami SS,Нейтральный,"после первой арки несколько разочаровался, ожи..."
1,Легенда о легендарных героях / Densetsu no Yuu...,Положительный,"Концовка какая-то мутная, ничего не понял. Над..."
2,Мобильный воин Гандам: Единорог / Kidou Senshi...,Положительный,"Что тут сказать? Никакого сравнени я с Seed, х..."
3,Невиданный цветок / Ano Hi Mita Hana no Namae ...,Положительный,Только что досмотрел и меня переполняют эмоции...
4,Принцесса-медуза / Kuragehime,Положительный,"И правда не оторваться. Но, по-моему, то, что ..."


In [5]:
df["text_len"] = df["text"].str.len()

print("Средняя длина:", round(df["text_len"].mean(), 2))
print("Медиана:", df["text_len"].median())
print("Мин/Макс:", df["text_len"].min(), df["text_len"].max())

print("\nБаланс классов:")
display(df["label"].value_counts())
display(df.head(3))


Средняя длина: 1302.74
Медиана: 864.0
Мин/Макс: 3 32767

Баланс классов:


label
Положительный    30073
Нейтральный      25263
Отрицательный    17880
Name: count, dtype: int64

,anime,label,text,text_len
0,Нежный укус / Amagami SS,Нейтральный,"после первой арки несколько разочаровался, ожи...",185
1,Легенда о легендарных героях / Densetsu no Yuu...,Положительный,"Концовка какая-то мутная, ничего не понял. Над...",148
2,Мобильный воин Гандам: Единорог / Kidou Senshi...,Положительный,"Что тут сказать? Никакого сравнени я с Seed, х...",897


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

vec = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    max_features=80000
)

X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

clf = LogisticRegression(max_iter=3000)
clf.fit(X_train_vec, y_train)

pred = clf.predict(X_test_vec)

print(classification_report(y_test, pred))
print("Confusion matrix:\n", confusion_matrix(y_test, pred))


               precision    recall  f1-score   support

  Нейтральный       0.48      0.43      0.46      5053
Отрицательный       0.67      0.61      0.64      3576
Положительный       0.64      0.73      0.68      6015

     accuracy                           0.60     14644
    macro avg       0.60      0.59      0.59     14644
 weighted avg       0.59      0.60      0.59     14644

Confusion matrix:
 [[2191  812 2050]
 [ 985 2164  427]
 [1381  253 4381]]


In [7]:
from pathlib import Path
Path("outputs").mkdir(exist_ok=True)

df.to_csv("outputs/anime_reviews_clean.csv", index=False, encoding="utf-8")

report = pd.DataFrame({"text": X_test, "true": y_test, "pred": pred})
report.to_csv("outputs/predictions_sample.csv", index=False, encoding="utf-8")

print("Saved: outputs/anime_reviews_clean.csv")
print("Saved: outputs/predictions_sample.csv")


Saved: outputs/anime_reviews_clean.csv
Saved: outputs/predictions_sample.csv
